In [2]:
import os
import numpy as np
import pandas as pd
from utils import load_sparse_csr

In [3]:
from sklearn.cross_validation import KFold
from sklearn import metrics
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import *
from sklearn import preprocessing
from sklearn.preprocessing import *

In [4]:
SEED_VAL = 200;
WORK_DIR = os.getcwd();
data_subset = "_10Percent"
YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
YELP_DATA_SPARSE_MATRIX_DIR = os.path.join(WORK_DIR, "data", "sparse_matrix")
YELP_DATA_SPARSE_MATRIX_DIR_FEAT_ADD = os.path.join(WORK_DIR, "data", "sparse_matrix_feat_add")
YELP_DATA_WORD_2_VEC_MODEL_DIR = os.path.join(WORK_DIR, "data", "word2vec_model")
YELP_DATA_WORD_2_VEC_MODEL_DIR_FEAT_ADD = os.path.join(WORK_DIR, "data", "word2vec_model_feat_add")

In [5]:
# Read data
read_filename = os.path.join(YELP_DATA_CSV_DIR, 'business_review_user' + data_subset + ".csv")
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

C:\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
def myNB(bow_feature_matrix_train, y_train, bow_feature_matrix_test, y_test):
    bernoulli_nb_classifier = BernoulliNB()
    
#     print np.isnan(bow_feature_matrix_train)
#     print np.isnan(y_train)
#     print np.isfinite(bow_feature_matrix_train)
#     print np.isfinite(y_train)

    bernoulli_nb_classifier.fit((bow_feature_matrix_train), y_train)
    
    bernoulli_nb_prediction = bernoulli_nb_classifier.predict(bow_feature_matrix_train)
    train_accuracy = metrics.accuracy_score(y_train, bernoulli_nb_prediction)
    
    bernoulli_nb_prediction2 = bernoulli_nb_classifier.predict(bow_feature_matrix_test)
    test_accuracy = metrics.accuracy_score(y_test, bernoulli_nb_prediction2)
                           
    return [train_accuracy, test_accuracy]

def myKFoldNB(X, y, n_fold):
    res_nb_train = []
    res_nb_test = []
    
    res_nb_train_avg = 0.0
    res_nb_test_avg = 0.0
    
    res_nb = []
    
    kf = KFold(X.shape[0], n_folds=n_fold)
    
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        res_nb = myNB(X_train, y_train, X_test, y_test)
        res_nb_train_avg = res_nb_train_avg + float(res_nb[0])
        res_nb_test_avg = res_nb_test_avg + float(res_nb[1])
        
        res_nb_train.append(res_nb[0])
        res_nb_test.append(res_nb[1])
        
    res_nb_train_avg = res_nb_train_avg / n_fold
    res_nb_test_avg = res_nb_test_avg / n_fold
    
    print "NB training data accuracy: " + str(res_nb_train)
    print "NB training data average accuracy: " + str(res_nb_train_avg)
    print "NB test data accuracy:" + str(res_nb_test)
    print "NB test data average accuracy: " + str(res_nb_test_avg)

y = np.array(df_data.review_stars.copy(), dtype='int64')

# Bag of Words

In [7]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR, "bagWords" + data_subset)
feature_matrix_bag_of_words = load_sparse_csr(spare_matrix_file + ".npz")

In [25]:
myKFoldNB(feature_matrix_bag_of_words, y, 5)

NB training data accuracy: [0.49455617119274004, 0.49375812299062866, 0.49432245707640743, 0.49367261782611671, 0.49524591285313635]
NB training data average accuracy: 0.494311056388
NB test data accuracy:[0.48883872585904187, 0.49508630321271407, 0.49018400711402971, 0.49273776136078618, 0.49150648699181432]
NB test data average accuracy: 0.491670656908


# Bag of Words + Hand crafted Features

In [26]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR_FEAT_ADD, "bagWords_feat_add" + data_subset)
feature_matrix_bag_of_words_and_hand_craft_features = load_sparse_csr(spare_matrix_file + ".npz")

In [27]:
myKFoldNB(feature_matrix_bag_of_words_and_hand_craft_features, y, 5)

NB training data accuracy: [0.55658731787399962, 0.55600588275531837, 0.55513943042159741, 0.55567526278587231, 0.556296600314659]
NB training data average accuracy: 0.55594089883
NB test data accuracy:[0.5078778758236997, 0.51510591239710879, 0.51145769204459945, 0.51104726725494221, 0.51216453473789814]
NB test data average accuracy: 0.511530656452


# Word embedding

In [10]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "word2vec_feature_matrix" + data_subset+ ".csv")
feature_matrix_word2vec = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')  

In [11]:
myKFoldNB(np.array(feature_matrix_word2vec,dtype = 'int64'), y, 5)

NB training data accuracy: [0.40869758533415418, 0.40598422144697538, 0.40733520304626397, 0.40671386551747724, 0.40672526620607885]
NB training data average accuracy: 0.40709122831
NB test data accuracy:[0.40066580021433296, 0.4115192557630481, 0.40611532936589367, 0.40860067948104067, 0.40855507672663427]
NB test data average accuracy: 0.40709122831


# Word 2 Vec + Hand crafted Features

In [12]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR_FEAT_ADD, "word2vec_add_feature_matrix" + data_subset+ ".csv")
feature_matrix_word2vec_and_hand_craft_features = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')  

In [13]:
myKFoldNB(np.array(feature_matrix_word2vec_and_hand_craft_features, dtype = 'int64'), y, 5)

NB training data accuracy: [0.46638506965820736, 0.46470916843377341, 0.46572953006361584, 0.46520509838794261, 0.46608295141026518]
NB training data average accuracy: 0.465622363591
NB test data accuracy:[0.462571539320975, 0.4692751442187108, 0.46519369769934105, 0.46729142440203386, 0.46378001231274368]
NB test data average accuracy: 0.465622363591


# Hand crafted features

In [14]:
feature_matrix_hand_craft_features = feature_matrix_word2vec_and_hand_craft_features[:,100:104]

In [15]:
myKFoldNB(feature_matrix_hand_craft_features, y, 5)

NB training data accuracy: [0.46638506965820736, 0.46470916843377341, 0.46572953006361584, 0.46520509838794261, 0.46608295141026518]
NB training data average accuracy: 0.465622363591
NB test data accuracy:[0.462571539320975, 0.4692751442187108, 0.46519369769934105, 0.46729142440203386, 0.46378001231274368]
NB test data average accuracy: 0.465622363591
